# Data Analysis of Flux Simulation Data

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import json
import os
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt

A numerical simulation creates datafile in the [Parquet file format](https://parquet.apache.org/documentation/latest/).  

In [ ]:
files = glob("data/*.parquet")
files

We can use `pd.read_parquet` to read the data and `pq.read_schema` to read the metadata.  For this set of simulations, the parameters `Pe` and `SS` are the only ones changing between simulations. We create a multidimensional Xarray dataset to hold the output from each experiment with the parameters serving as dimensions.

In [ ]:
def load_parquet_data(file):
    # load parquet file as pandas dataframe
    df = pd.read_parquet(file)
    # remove multi-index on column
    df = df.droplevel(level=0, axis=1)
    # convert to a Xarray Dataset
    ds = xr.Dataset(df)

    # metadata is also stored in the parquet file
    def DictFilter(x,y):
        """Slice dict. Usage: dictfilter(originaldict, [dictkeystokeep])"""
        return dict([ (i,x[i]) for i in x if i in set(y)])

    schema = pq.read_schema(file)
    metadata = json.loads(schema.metadata[b'metadata'])

    mdata = metadata['updatedparameters'][0] if 'updatedparameters' in metadata.keys() else metadata['yaml'][0]
    mdata = DictFilter(mdata, ['Mesh','Parameters'])
    mdata.update({'Simulation': metadata['simulation']})

    mesh_metadata = mdata['Mesh']
    parameters_metadata = mdata['Parameters']
    simulation_metadata = mdata['Simulation']

    # Pull out variables that are changing between experiments
    Pe = parameters_metadata['Pe']
    SS = mesh_metadata['SS']

    # Add new dimensions for the changing parameters
    ds = ds.expand_dims(['Pe', 'SS'])
    ds['Pe'] = [Pe]
    ds['SS'] = [SS]

    # remove those parameters from the shared list of parameters
    del parameters_metadata['Pe']
    del mesh_metadata['SS']
    
    # nx: None will not be storable in a NetCDF file. Remove it.
    if 'nx' in mesh_metadata and mesh_metadata['nx'] is None:
        del mesh_metadata['nx']
    # NetCDF does not support boolean for attribute data type
    if 'override' in mesh_metadata:
        mesh_metadata['override'] = int(mesh_metadata['override'])
        
    ds = ds.assign_attrs(mesh_metadata)
    ds = ds.assign_attrs(parameters_metadata)
    ds = ds.assign_attrs(simulation_metadata)
    
    return ds

As an example, here one of the data files transformed into a Xarray dataset.

In [ ]:
ds = load_parquet_data(files[0])
ds

These datasets can be merged into one large dataset.

In [ ]:
ds = xr.merge([load_parquet_data(f) for f in files], combine_attrs='drop_conflicts') # combine_attrs='drop_conflicts' so that only common metadata is shown for collection of datasets
ds

In [ ]:
# When analyzing data it is important to know where each subset of the dataset comes from
# That is, we need to be able to extract metadata for each individual simulation
# Notice "Attributes" above is for the entire collection.
# The problem is that we lost the attributes of individual datasets when we called xr.merge().

# Example: select a single simulation from the complete dataset.
# Any attributes that are unique to the data, such as 'timestamp' were lost.

print('ACTUAL:', ds.sel(Pe=200, SS=1).attrs) # extracted from merged dataset
print('EXPECTED:',load_parquet_data(files[0]).attrs) # from single dataset

The NaNs are occuring because some simulations stopped earlier than others. This can be fixed by filling or padding the data.
We can store the complete dataset as a single NetCDF file.

In [ ]:
if os.path.exists("data.nc"):
    os.remove("data.nc")
    
ds.to_netcdf("data.nc")

For future analysis, we can just load this NetCDF file directly and skip the building of the NetCDF file.

In [ ]:
ds = xr.open_dataset("data.nc")
ds

We can select the data from only one experiment by indexing by `Pe` or `SS`:

In [ ]:
ds.sel(Pe=200, SS=1).plot.scatter(x="Time", y="s1", label='s1')
ds.sel(Pe=200, SS=1).plot.scatter(x="Time", y="s2", label='s2')
plt.legend(loc='lower right')
plt.ylabel('Dimensional Flux')
plt.show()

But the real power comes with using `Pe` and `SS` as part of the dataset:

In [ ]:
ds.plot.scatter(x='Time', y='s1', hue='Pe', col='SS')
ds.plot.scatter(x='Time', y='s2', hue='Pe', col='SS')
plt.show()

Or grouping by Pe

In [ ]:
ds.plot.scatter(x='Time', y='s1', hue='SS', col='Pe')
ds.plot.scatter(x='Time', y='s2', hue='SS', col='Pe')
plt.show()

To summarize over all Pe and SS, we can calculate the maximum flux in Time

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))
ds.s1.max(dim='Time').plot(ax=ax[0])
ax[0].set_title('s1')
ds.s2.max(dim='Time').plot(ax=ax[1])
plt.show()

If you have the Holoviz package installed (hvplot) you can get some instant interactivity:

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import hvplot.xarray

Use the slider to change the value of SS:

In [ ]:
s1 = ds.hvplot.scatter(x='Time', y='s1', by='Pe')
s2 = ds.hvplot.scatter(x='Time', y='s2', by='Pe')
(s1+s2).cols(1)

Or the equilibrium fluxes:

In [ ]:
equlibrium = ds.max("Time")
s1 = equlibrium.hvplot.heatmap(x='SS', y='Pe', C='s1')
s2 = equlibrium.hvplot.heatmap(x='SS', y='Pe', C='s2') 

(s1+s2).cols(1)